In [1]:
#numpy and pandas for data manipulation
import numpy as np
import pandas as pd

#sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

In [2]:
samples = pd.read_csv('data/application_train.csv')
train_label = samples['TARGET']
train_data = samples.drop(columns=['TARGET'])

In [3]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train_data:
    if train_data[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train_data[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_data[col])
            # Transform 
            train_data[col] = le.transform(train_data[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [4]:
# one-hot encoding of categorical variables
train_data = pd.get_dummies(train_data)

In [5]:
# # Create an anomalous flag column
train_data['DAYS_EMPLOYED_ANOM'] = train_data["DAYS_EMPLOYED"] == 365243

# # Replace the anomalous values with nan
train_data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer


# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train_data)

# Transform both training and testing data
train_data = imputer.transform(train_data)

# Repeat with the scaler
scaler.fit(train_data)
train_data = scaler.transform(train_data)

print('Training data shape: ', train_data.shape)

Training data shape:  (307511, 243)


In [7]:
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_predict

# Make the model with the specified regularization parameter
# log_reg =

# 80-20 train-test case
kf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
log_reg = LogisticRegression(C = 0.01, max_iter=500, solver='sag', fit_intercept=True)
y_pred = cross_val_predict(estimator=log_reg , X=train_data, y=train_label, cv=kf)
from sklearn import metrics

print(metrics.confusion_matrix(train_label, y_pred))
print(metrics.classification_report(train_label ,y_pred))
print("roc - {}".format(metrics.roc_auc_score(train_label, y_pred)))

[[282576    110]
 [ 24723    102]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96    282686
           1       0.48      0.00      0.01     24825

    accuracy                           0.92    307511
   macro avg       0.70      0.50      0.48    307511
weighted avg       0.88      0.92      0.88    307511

roc - 0.5018598185002724
